In [ ]:
using Catalyst, DifferentialEquations, Distributions, CairoMakie

In [ ]:
CN_distribution = truncated(Gamma(0.57, 120))

support = range(-5,300, length=300)
lines(support, pdf.(CN_distribution, support); axis=(; title="Copy number distribution", xlabel="CN", ylabel="P[CN]"))

In [ ]:
single_iFFL_rn = @reaction_network single_transcript_iFFL begin
    @parameters begin
        α_im=4.67e-2 #Transcription rate constant [1/s]
        δ_im=2.88e-4 #Immature mRNA degradation rate constant [1/s]
        δ_m=2.88e-4 #Mature mRNA degradation rate constant [1/s]
        δ_mi=2.88e-4 #miRNA degradation rate constant [1/s]
        δ_p=9.67e-5 #Protein degradation rate constant [1/s] 
        α_p=3.33e-4 #Translation rate constant [1/s]
        ζ=0.0 #Translation efficiency for RISC complex [unitless], Range: 0 to 1
        r_splicing=2.0e-3 #Splicing rate constant [1/s]
        r_drosha=1e-2 #pri-miRNA processing rate constant [1/s] 
        r_dicer=1e-3 #pre-miRNA processing rate constant [1/s]
        k_miRNA_bind=1e-5 #miRNA-RISC binding rate constant [1/mol*s]
        k_miRNA_deg=2.16e-5 #miRNA-RISC unbinding rate constant [1/s]
        k_mRNA_bind=1.84e-6 #RISC complex formation rate constant[1/mol*s]
        k_mRNA_unbind=0.303 #RISC complex deformation rate constant [1/s]
        k_deg=7e-3 #RISC-mediated mRNA degradation rate constant [1/s]
    end
    
    # Transcription
    α_im, gene --> immature_mRNA + gene #Set copy number as the initial number of "gene"
    α_im, gene --> ctl_mRNA + gene
    # Degradation of products
    δ_im, immature_mRNA --> ∅
    δ_m, mRNA --> ∅
    δ_m, ctl_mRNA --> ∅
    δ_mi, miRNA --> ∅
    δ_p, protein --> ∅
    δ_p, ctl_protein --> ∅
    # Production of protein
    α_p, mRNA --> protein + mRNA
    α_p, ctl_mRNA --> ctl_protein + ctl_mRNA
    α_p * ζ, risc_miRNA_mRNA --> protein + risc_miRNA_mRNA
    # Splicing
    r_splicing, immature_mRNA --> mRNA + pri_miRNA
    # miRNA processing
    r_drosha, pri_miRNA --> pre_miRNA # drosha
    r_dicer, pre_miRNA --> miRNA # dicer
    # Knockdown
    k_miRNA_bind, risc + miRNA --> risc_miRNA
    k_miRNA_deg, risc_miRNA --> risc
    (k_mRNA_bind, k_mRNA_unbind), risc_miRNA + mRNA <--> risc_miRNA_mRNA
    k_deg, risc_miRNA_mRNA --> risc_miRNA
end

In [ ]:
species(single_iFFL_rn)

In [ ]:
u0 = [:gene => 100, :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
dprob = DiscreteProblem(single_iFFL_rn, u0, tspan)
jprob = JumpProblem(single_iFFL_rn, dprob, Direct(), save_positions=(false,false))
sol = solve(jprob, SSAStepper(), saveat=1000.)

In [ ]:
plot(sol[6,:], label="regulated gene";    
    figure = (; size = (400, 400)),
    axis = (; title = "stochastic simulation", xlabel = "time", ylabel = "protein expression"))
plot!(sol[7,:], label="unregulated gene")
axislegend(position=:lt)
current_figure()

In [ ]:
CN = floor(rand(CN_distribution))
u0 = [:gene => CN, :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
dprob = DiscreteProblem(single_iFFL_rn, u0, tspan)
jprob = JumpProblem(single_iFFL_rn, dprob, Direct(), save_positions=(false,false))
sol = solve(jprob, SSAStepper(), saveat=1000.)

In [ ]:
plot(sol[6,:], label="regulated gene";    
    figure = (; size = (400, 400)),
    axis = (; title = "stochastic simulation CN="*string(CN), xlabel = "time", ylabel = "protein expression"))
plot!(sol[7,:], label="unregulated gene")
axislegend(position=:lt)
current_figure()

In [ ]:
function prob_func(prob,i,repeat)
    remake(prob, u0 = [:gene => floor(rand(CN_distribution)), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function output_func(sol, i)
    [sol[:,6:7]], false
end

u0 = [:gene => floor(rand(CN_distribution)), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 100000.)
jprob = JumpProblem(single_iFFL_rn, DiscreteProblem(single_iFFL_rn, u0, tspan), Direct(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func)
solns = solve(eprob, SSAStepper(); trajectories=50, saveat=1000.)

In [ ]:
figure = Figure(size = (600, 400))
axis = Axis(figure[1, 1], title = "stochastic simulations", xlabel = "time", ylabel = "protein expression")
for sol in solns
    lines!(axis, sol[6,:], color=(:blue, 0.5), label="regulated gene")
    lines!(axis, sol[7,:], color=(:gray, 0.5), label="unregulated gene")
end
current_figure()

In [ ]:
function prob_func(prob,i,repeat)
    remake(prob, u0 = [:gene => floor(rand(CN_distribution)), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0])
end

function output_func(sol, i)
    [sol.u[end,:]], false
end

u0 = [:gene => floor(rand(CN_distribution)), :risc => 10000, :immature_mRNA => 0, :ctl_mRNA => 0, :miRNA => 0, :mRNA => 0, :protein => 0, :ctl_protein => 0, :pri_miRNA => 0, :pre_miRNA => 0, :risc_miRNA => 0, :risc_miRNA_mRNA => 0]
tspan = (0., 10000.)
jprob = JumpProblem(single_iFFL_rn, DiscreteProblem(single_iFFL_rn, u0, tspan), Direct(), save_positions=(false,false))
eprob = EnsembleProblem(jprob, prob_func=prob_func, output_func=output_func, safetycopy=false)
solns = solve(eprob, SSAStepper(), EnsembleThreads(); trajectories=1000)

In [ ]:
reg_vec = zeros(1000)
unreg_vec = zeros(1000)
cn_vec = zeros(1000)
for j = 1:lastindex(solns)
    reg_vec[j] = solns[j][1][1][6]
    unreg_vec[j] = solns[j][1][1][7]
    cn_vec[j] = solns[j][1][1][1]
end
display(cn_vec)

In [ ]:
f = Figure()
ax1 = Axis(f[1,1],title="Single transcript iFFL: 10,000 RISC", xlabel = "Copy Number", ylabel = "Protein Expression",limits = (nothing, nothing, -10, 150000))
s1 = scatter!(ax1, cn_vec,unreg_vec, color=(:gray, 0.5), label="unregulated gene")
s2 = scatter!(ax1, cn_vec,reg_vec, color=(:blue, 0.5), label="regulated gene")
axislegend(position=:lt)
ax2 = Axis(f[1,2],limits = (nothing, nothing, -10, 150000))
linkyaxes!(ax1,ax2)
d1 = density!(unreg_vec, color=(:gray, 0.5), direction = :y)
d2 = density!(reg_vec, color=(:blue, 0.5), direction = :y)
xlims!(ax2, low = 0)
hidedecorations!(ax2, grid = false)
colsize!(f.layout, 1, Auto(3))
save("jl_output/2024.05.08_modeling/single_transcript_10000_risc.svg",f)
current_figure()